In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import seaborn as sns
import warnings
from sklearn.cluster import KMeans
from kneed import KneeLocator
import collections
import auxiliary_functions
import pprint
import json
import random
from itertools import combinations

In [3]:
data = pd.read_csv("./data_house/database.csv", sep = ',') 
column_names = data.columns
n = len(data.columns)
print("Dataset shape:", data.shape)

# Generate a random query   
m = random.randint(1,4)
df = data.sample(n = m, axis = 'columns').sample()
row = []
df_fake_queries = pd.DataFrame(index = range(1), columns = column_names)
df_fake_queries.drop(df_fake_queries.columns[df_fake_queries.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

for j in range(len(df.columns)):
    row.append(''.join((str(df.columns[j]),'=',str(df.iloc[0][j]))))
    df_fake_queries[str(df.columns[j])].iloc[0] = df.iloc[0][j]

print('Query: ', row)
print('Dataframe of query: ', df_fake_queries)

Dataset shape: (10000, 13)
Query:  ['garden_sm=8', 'floors=2', 'price=25', 'doors=10']
Dataframe of query:    nrooms nbedrooms nbath   sm garden_sm floors gargae_sm price year windows  \
0    NaN       NaN   NaN  NaN         8      2       NaN    25  NaN     NaN   

  dist_city doors  
0       NaN    10  


In [4]:
query = df_fake_queries.dropna(axis = 1)
query_columns = query.columns
query_values = query.values[0]
print(query)

  garden_sm floors price doors
0         8      2    25    10


In [5]:
# We have to check if the query already exists in our query database
queries =  pd.read_csv("./data_house/queries_to_use.csv", sep = ',', index_col = 0)

In [6]:
resData = queries.merge(df_fake_queries, how = 'inner' ,indicator=False)
print("Common rows between two DataFrames...\n",resData)

Common rows between two DataFrames...
 Empty DataFrame
Columns: [query_id, nrooms, nbedrooms, nbath, sm, garden_sm, floors, gargae_sm, price, year, windows, dist_city, doors]
Index: []


In [7]:
comb, length = auxiliary_functions.combination(query_columns)
comb, length

(['garden_sm',
  'floors',
  'price',
  'doors',
  ('garden_sm', 'floors'),
  ('garden_sm', 'price'),
  ('garden_sm', 'doors'),
  ('floors', 'price'),
  ('floors', 'doors'),
  ('price', 'doors'),
  ('garden_sm', 'floors', 'price'),
  ('garden_sm', 'floors', 'doors'),
  ('garden_sm', 'price', 'doors'),
  ('floors', 'price', 'doors'),
  ('garden_sm', 'floors', 'price', 'doors')],
 15)

In [8]:
dict_query = {}
# We create a dictionary with the possible combinations:
comb, l = auxiliary_functions.combination(query_columns)
for i in range(l):
    dict_query.update({str(comb[i]) : []} )
print(dict_query)   
        

{'garden_sm': [], 'floors': [], 'price': [], 'doors': [], "('garden_sm', 'floors')": [], "('garden_sm', 'price')": [], "('garden_sm', 'doors')": [], "('floors', 'price')": [], "('floors', 'doors')": [], "('price', 'doors')": [], "('garden_sm', 'floors', 'price')": [], "('garden_sm', 'floors', 'doors')": [], "('garden_sm', 'price', 'doors')": [], "('floors', 'price', 'doors')": [], "('garden_sm', 'floors', 'price', 'doors')": []}


In [9]:
# We can look for queries that share some of the values
# We create a dictionary to update the repeated values:
length = len(query_columns)

dict_query = auxiliary_functions.matching_queries(length, query_columns, query, dict_query, queries)

Case 4: up to 4 common value
Dictionary:  {'garden_sm': [47, 76, 122, 138, 169, 172, 175, 190, 218, 284, 318, 358, 374, 383, 396, 550, 569, 737, 991, 1027, 1110, 1113, 1132, 1166, 1252, 1357, 1449, 1562, 1574, 1604, 1646, 1787, 1854, 1920, 1991], 'floors': [1, 20, 65, 66, 67, 68, 71, 104, 111, 150, 155, 159, 173, 182, 212, 218, 227, 233, 243, 254, 255, 297, 310, 312, 315, 336, 338, 349, 363, 368, 371, 390, 406, 413, 417, 429, 431, 462, 478, 483, 502, 506, 512, 554, 559, 578, 591, 599, 614, 617, 632, 637, 676, 683, 715, 735, 739, 743, 775, 781, 802, 830, 842, 850, 855, 866, 883, 895, 901, 902, 917, 946, 952, 954, 961, 1012, 1014, 1020, 1026, 1037, 1044, 1071, 1095, 1098, 1123, 1132, 1146, 1163, 1179, 1226, 1241, 1267, 1278, 1288, 1305, 1319, 1329, 1346, 1349, 1365, 1403, 1420, 1421, 1423, 1444, 1446, 1449, 1456, 1508, 1539, 1542, 1545, 1548, 1562, 1567, 1580, 1599, 1612, 1621, 1622, 1684, 1701, 1740, 1749, 1772, 1775, 1799, 1809, 1810, 1812, 1861, 1865, 1883, 1901, 1913, 1946, 1949, 197

c:\Users\elsal\OneDrive\Documentos\Master\EIT\Trento\DataMining\project\auxiliary_functions.py:258: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  idx = list(queries[queries[str(query_columns[j])] == query.iloc[0,j]][queries[str(query_columns[i])] == query.iloc[0,i]].index)
c:\Users\elsal\OneDrive\Documentos\Master\EIT\Trento\DataMining\project\auxiliary_functions.py:265: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  idx = list(queries[queries[str(query_columns[j])] == query.iloc[0,j]][queries[str(query_columns[i])] == query.iloc[0,i]][queries[queries[str(query_columns[k])] == query.iloc[0,k]]].index)
c:\Users\elsal\OneDrive\Documentos\Master\EIT\Trento\DataMining\project\auxiliary_functions.py:270: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  idx = list(queries[queries[str(query_columns[0])] == query.iloc[0,0]][queries[str(query_columns[1])] == query.iloc[0,1]][queries[str(query_columns[2])